In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import re
import pycountry
import openpyxl
import random

df = pd.read_csv('../../data/prokanon.combs4.csv')
df.loc[(df['mau'] == 1000), 'mau'] = df['dau']
df['mau'] = df['mau'].apply(lambda x: x if x >= 83 else random.randint(1, 83))
df.sort_values(by='mau',inplace=True)


df['cumsum'] = df.groupby('country')['mau'].cumsum()

sum_per_country = df.groupby('country')['mau'].sum()

df['cdf'] = df['cumsum'] / df['country'].map(sum_per_country)

paises = df[['country']].drop_duplicates()

df

,Unnamed: 0,age,country,gender,rel_status,mau,dau,percentage,cdf_true,cumsum,cdf
81317,135797,39,BS,2,10,1,0,4.153620e-07,0.033776,1.0,0.000003
35878,179010,38,GQ,2,2,1,0,4.153620e-07,0.014903,1.0,0.000004
55845,161295,45,VU,2,1,1,0,4.153620e-07,0.023196,1.0,0.000004
15750,168667,38,CV,1,10,1,0,4.153620e-07,0.006542,1.0,0.000002
55847,161287,45,VU,1,13,1,0,4.153620e-07,0.023197,2.0,0.000008
...,...,...,...,...,...,...,...,...,...,...,...
216195,66809,21,IN,1,6,10100000,6684350,4.195157e-03,0.980603,352645535.0,0.883059
216196,66829,22,IN,1,6,10300000,6934642,4.278229e-03,0.984881,362945535.0,0.908851
216197,66889,25,IN,1,6,10400000,7407028,4.319765e-03,0.989201,373345535.0,0.934893
216198,66949,28,IN,1,6,12300000,8671680,5.108953e-03,0.994310,385645535.0,0.965694


In [2]:
df[df['mau'] <=1000]

,Unnamed: 0,age,country,gender,rel_status,mau,dau,percentage,cdf_true,cumsum,cdf
81317,135797,39,BS,2,10,1,0,4.153620e-07,0.033776,1.0,0.000003
35878,179010,38,GQ,2,2,1,0,4.153620e-07,0.014903,1.0,0.000004
55845,161295,45,VU,2,1,1,0,4.153620e-07,0.023196,1.0,0.000004
15750,168667,38,CV,1,10,1,0,4.153620e-07,0.006542,1.0,0.000002
55847,161287,45,VU,1,13,1,0,4.153620e-07,0.023197,2.0,0.000008
...,...,...,...,...,...,...,...,...,...,...,...
35492,100257,48,CL,2,10,999,999,4.153620e-07,0.014742,296136.0,0.022973
35481,100047,38,CL,1,10,999,999,4.153620e-07,0.014738,297135.0,0.023051
83608,26821,43,IS,1,3,999,999,4.153620e-07,0.034728,391867.0,0.906539
133718,76836,52,LB,2,1,999,999,4.153620e-07,0.055542,236806.0,0.067311


In [3]:
print(105043/len(df[df['mau'] <=1000]))

0.6457190980845361


In [4]:
df['dau'].value_counts().sort_index()

dau
0          105043
83              2
87              1
90              2
91              2
            ...  
7093144         1
7180640         1
7407028         1
8671680         1
9281285         1
Name: count, Length: 23466, dtype: int64

In [5]:

res_geq090 = df[df['cdf'] >= 0.1]

idx_min = res_geq090.groupby('country')['cdf'].apply(lambda x: (x - 0.1).abs().idxmin())

results_90 = res_geq090.loc[idx_min]



res_geq095 = df[df['cdf'] >= 0.05]

idx_min = res_geq095.groupby('country')['cdf'].apply(lambda x: (x - 0.05).abs().idxmin())

results_95 = res_geq095.loc[idx_min]



res_geq099 = df[df['cdf'] >= 0.01]

idx_min = res_geq099.groupby('country')['cdf'].apply(lambda x: (x - 0.01).abs().idxmin())

results_99 = res_geq099.loc[idx_min]

In [6]:



for index, row in results_90[['country','mau']].iterrows():
    mask = paises['country'] == row['country']
    paises.loc[mask,'K^4_90'] = row['mau']


for index, row in results_95[['country','mau']].iterrows():
    mask = paises['country'] == row['country']
    paises.loc[mask,'K^4_95'] = row['mau']


for index, row in results_99[['country','mau']].iterrows():
    mask = paises['country'] == row['country']
    paises.loc[mask,'K^4_99'] = row['mau']



for index, value in sum_per_country.items():
    mask = paises['country'] == index
    paises.loc[mask,'RK^4_90'] = (paises.loc[mask,'K^4_90']/value) * 100 
    paises.loc[mask,'RK^4_95'] = (paises.loc[mask,'K^4_95']/value) * 100 
    paises.loc[mask,'RK^4_99'] = (paises.loc[mask,'K^4_99']/value) * 100 



print(len(df.index))

216200


In [7]:
paises = paises.sort_values(by='country',ascending=True).dropna()
paises['K^4_90'] = paises['K^4_90'].astype('int')
paises['K^4_95'] = paises['K^4_95'].astype('int')
paises['K^4_99'] = paises['K^4_99'].astype('int')
paises['RK^4_90'] = paises['RK^4_90'].round(3)
paises['RK^4_95'] = paises['RK^4_95'].round(3)
paises['RK^4_99'] = paises['RK^4_99'].round(3)



paises.to_parquet('../../data/analyzed/fb/FB_K^4_X_by_country.parquet')
paises

,country,K^4_90,K^4_95,K^4_99,RK^4_90,RK^4_95,RK^4_99
102785,AD,68,49,21,0.032,0.023,0.010
131707,AE,8300,3000,798,0.085,0.031,0.008
159876,AF,7300,1900,497,0.178,0.046,0.012
18031,AG,74,51,22,0.033,0.023,0.010
102293,AL,989,887,68,0.074,0.066,0.005
...,...,...,...,...,...,...,...
39040,YE,4900,1100,497,0.125,0.028,0.013
129852,YT,73,51,23,0.031,0.022,0.010
112048,ZA,21300,8400,1600,0.082,0.032,0.006
107758,ZM,2500,989,497,0.069,0.027,0.014
